In [ ]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
os.environ['MUJOCO_PY_MUJOCO_PATH']='/scratch_net/biwidl211/rl_course_10/.mujoco/mujoco210'

#add DiffuseDrive to path. Dont know why, but else diffuser folder is not seen.
# os.environ['PYTHONPATH'] = str(os.environ['PYTHONPATH']) + '~/DiffuseDrive'

import diffuser.utils as utils
import torch
import wandb
# logging.basicConfig(filename='logs/training.log', encoding='utf-8', level=logging.INFO)
from logger_module import logger
from tqdm import tqdm

In [ ]:
dataset_config = utils.Config(
    'datasets.CollectedSequenceDataset',
    savepath='dataset_config.pkl',
    env='carla-expert',
    horizon=12,
    normalizer='CDFNormalizer',
    preprocess_fns=[],
    use_padding=True,
    max_path_length=10,
    include_returns=True,
    returns_scale=400,
    discount=0.99,
    termination_penalty=-100,
    past_image_cond = False
)


dataset = dataset_config()

In [ ]:
def cycle(dl):
    while True:
        for data in dl:
            yield data

In [ ]:
dataloader = cycle(torch.utils.data.DataLoader(
    dataset, batch_size=1, num_workers=0, pin_memory=True, shuffle = False
))

In [ ]:
tmp = next(dataloader)


In [ ]:
print(tmp[0])
print(tmp[0][0,:,:-1]) # x and y coordinates

In [ ]:
all_zeros_count = 0
for i, data in enumerate(tqdm(dataloader)):
    # print(i)
    # print(data[0].max())
    # print(data[0])
    if data[0][0,:,:-1].max() <= 1e-6:
        all_zeros_count += 1
    
    if i > 50000:
        break

In [ ]:
print(all_zeros_count)
print(i)

In [ ]:
print(all_zeros_count/(i+1))